In [ ]:
import pandas as pd
import numpy as np
import requests

url = "https://en.wikipedia.org/wiki/2009_MotoGP_World_Championship"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)

len(tables)

/tmp/ipython-input-863199352.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


32

In [ ]:
tables[0].head()


,Round,Date,Grand Prix,Circuit
0,1,12–13 April ‡,Commercialbank Grand Prix of Qatar,Losail International Circuit
1,2,26 April,Polini Grand Prix of Japan,Twin Ring Motegi
2,3,3 May,Gran Premio bwin.com de España,Circuito de Jerez
3,4,17 May,Grand Prix de France,Bugatti Circuit
4,5,31 May,Gran Premio d'Italia Alice,Mugello Circuit


In [ ]:
tables[1].head()

,Round,Date,Grand Prix,Circuit,125cc winner,250cc winner,MotoGP winner,Report
0,1,12–13 April ‡,Qatar motorcycle Grand Prix,Losail,Andrea Iannone,Héctor Barberá,Casey Stoner,Report
1,2,26 April,Japanese motorcycle Grand Prix,Motegi,Andrea Iannone,Álvaro Bautista,Jorge Lorenzo,Report
2,3,3 May,Spanish motorcycle Grand Prix,Jerez,Bradley Smith,Hiroshi Aoyama,Valentino Rossi,Report
3,4,17 May,French motorcycle Grand Prix,Le Mans,Julián Simón,Marco Simoncelli,Jorge Lorenzo,Report
4,5,31 May,Italian motorcycle Grand Prix,Mugello,Bradley Smith,Mattia Pasini,Casey Stoner,Report


In [ ]:
tables[2].head()

,0,1,2,3,4,5
0,Team Constructor Motorcycle No. Rider Rounds ...,Key Regular rider Wildcard rider Replacement r...,NaN,NaN,NaN,NaN
1,Team,Constructor,Motorcycle,No.,Rider,Rounds
2,Ducati Marlboro Team Ducati Team (rd 8 and 12),Ducati,Desmosedici GP9,27,Casey Stoner[29],"1–10, 14–17"
3,Ducati Marlboro Team Ducati Team (rd 8 and 12),Ducati,Desmosedici GP9,36,Mika Kallio[30],11–13
4,Ducati Marlboro Team Ducati Team (rd 8 and 12),Ducati,Desmosedici GP9,69,Nicky Hayden[31],All


In [ ]:
df = tables[1].copy()
df.head()

df = df[["Grand Prix", "Circuit", "MotoGP winner"]]
df.head()

df.columns = ["grand_prix", "circuit", "winner"]
df.head()


,grand_prix,circuit,winner
0,Qatar motorcycle Grand Prix,Losail,Casey Stoner
1,Japanese motorcycle Grand Prix,Motegi,Jorge Lorenzo
2,Spanish motorcycle Grand Prix,Jerez,Valentino Rossi
3,French motorcycle Grand Prix,Le Mans,Jorge Lorenzo
4,Italian motorcycle Grand Prix,Mugello,Casey Stoner


In [ ]:
df["is_winner"] = 1


In [ ]:
top_riders = [
    "Valentino Rossi",
    "Jorge Lorenzo",
    "Casey Stoner",
    "Dani Pedrosa",
    "Andrea Dovizioso"
]


In [ ]:
rows = []

for _, row in df.iterrows():
    for rider in top_riders:
        if rider != row["winner"]:
            rows.append({
                "grand_prix": row["grand_prix"],
                "circuit": row["circuit"],
                "rider": rider,
                "is_winner": 0
            })


In [ ]:
winner_rows = df.copy()
winner_rows["rider"] = winner_rows["winner"]
winner_rows["is_winner"] = 1

winner_rows = winner_rows[["grand_prix", "circuit", "rider", "is_winner"]]


In [ ]:
non_winners_df = pd.DataFrame(rows)
final_df = pd.concat([winner_rows, non_winners_df], ignore_index=True)

final_df.head()


,grand_prix,circuit,rider,is_winner
0,Qatar motorcycle Grand Prix,Losail,Casey Stoner,1
1,Japanese motorcycle Grand Prix,Motegi,Jorge Lorenzo,1
2,Spanish motorcycle Grand Prix,Jerez,Valentino Rossi,1
3,French motorcycle Grand Prix,Le Mans,Jorge Lorenzo,1
4,Italian motorcycle Grand Prix,Mugello,Casey Stoner,1


In [ ]:
final_df["is_winner"].value_counts()


,count
is_winner,
0,68
1,17


In [ ]:
#NEXT STEP = LOGISTIC REGRESSION (BASELINE MODEL)
X = final_df[["grand_prix", "circuit", "rider"]]
y = final_df["is_winner"]


In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded.head()


,grand_prix_British motorcycle Grand Prix,grand_prix_Catalan motorcycle Grand Prix,grand_prix_Czech Republic motorcycle Grand Prix,grand_prix_Dutch TT,grand_prix_French motorcycle Grand Prix,grand_prix_German motorcycle Grand Prix,grand_prix_Indianapolis motorcycle Grand Prix,grand_prix_Italian motorcycle Grand Prix,grand_prix_Japanese motorcycle Grand Prix,grand_prix_Malaysian motorcycle Grand Prix,...,circuit_Motegi,circuit_Mugello,circuit_Phillip Island,circuit_Sachsenring,circuit_Sepang,circuit_Valencia,rider_Casey Stoner,rider_Dani Pedrosa,rider_Jorge Lorenzo,rider_Valentino Rossi
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,True,False,False,False


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8235294117647058
[[14  0]
 [ 3  0]]
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       0.00      0.00      0.00         3

    accuracy                           0.82        17
   macro avg       0.41      0.50      0.45        17
weighted avg       0.68      0.82      0.74        17



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
y_pred_bal = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred_bal))
print(confusion_matrix(y_test, y_pred_bal))
print(classification_report(y_test, y_pred_bal))


Accuracy: 0.5882352941176471
[[10  4]
 [ 3  0]]
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        14
           1       0.00      0.00      0.00         3

    accuracy                           0.59        17
   macro avg       0.38      0.36      0.37        17
weighted avg       0.63      0.59      0.61        17



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)

rf_model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [ ]:
y_pred_rf = rf_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Accuracy: 0.8235294117647058
[[14  0]
 [ 3  0]]
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       0.00      0.00      0.00         3

    accuracy                           0.82        17
   macro avg       0.41      0.50      0.45        17
weighted avg       0.68      0.82      0.74        17



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#NEXT STEP = XGBOOST

from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    scale_pos_weight=4,  # imbalance handle
    random_state=42,
    eval_metric="logloss"
)

xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


Accuracy: 0.5882352941176471
[[10  4]
 [ 3  0]]
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        14
           1       0.00      0.00      0.00         3

    accuracy                           0.59        17
   macro avg       0.38      0.36      0.37        17
weighted avg       0.63      0.59      0.61        17



In [ ]:
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_prob_xgb


array([0.15499571, 0.00990715, 0.7516389 , 0.09488183, 0.58365446,
       0.09488183, 0.02249752, 0.0250676 , 0.07001621, 0.42548496,
       0.22512679, 0.02538381, 0.862903  , 0.3708322 , 0.8990721 ,
       0.18228373, 0.24458326], dtype=float32)

In [ ]:
import numpy as np

y_pred_custom = (y_prob_xgb >= 0.3).astype(int)


In [ ]:
print(confusion_matrix(y_test, y_pred_custom))
print(classification_report(y_test, y_pred_custom))


[[8 6]
 [3 0]]
              precision    recall  f1-score   support

           0       0.73      0.57      0.64        14
           1       0.00      0.00      0.00         3

    accuracy                           0.47        17
   macro avg       0.36      0.29      0.32        17
weighted avg       0.60      0.47      0.53        17



In [ ]:
import os

os.listdir("/content")


['.config', 'sample_data']

In [45]:
final_df.head()


,grand_prix,circuit,rider,is_winner
0,Qatar motorcycle Grand Prix,Losail,Casey Stoner,1
1,Japanese motorcycle Grand Prix,Motegi,Jorge Lorenzo,1
2,Spanish motorcycle Grand Prix,Jerez,Valentino Rossi,1
3,French motorcycle Grand Prix,Le Mans,Jorge Lorenzo,1
4,Italian motorcycle Grand Prix,Mugello,Casey Stoner,1


In [46]:
final_df.to_csv("/content/motogp_2009_predictions.csv", index=False)


In [47]:
os.listdir("/content")


['.config', 'motogp_2009_predictions.csv', 'sample_data']

In [48]:
from google.colab import files
files.download("/content/motogp_2009_predictions.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
final_df["win_probability"] = xgb_model.predict_proba(X_encoded)[:, 1]

final_df.head()


,grand_prix,circuit,rider,is_winner,win_probability
0,Qatar motorcycle Grand Prix,Losail,Casey Stoner,1,0.674556
1,Japanese motorcycle Grand Prix,Motegi,Jorge Lorenzo,1,0.094882
2,Spanish motorcycle Grand Prix,Jerez,Valentino Rossi,1,0.805551
3,French motorcycle Grand Prix,Le Mans,Jorge Lorenzo,1,0.680167
4,Italian motorcycle Grand Prix,Mugello,Casey Stoner,1,0.758316


In [51]:
final_df.to_csv("/content/motogp_2009_predictions.csv", index=False)


In [52]:
import os
os.listdir("/content")


['.config', 'motogp_2009_predictions.csv', 'sample_data']

In [53]:
from google.colab import files
files.download("/content/motogp_2009_predictions.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>